<a href="https://colab.research.google.com/github/Rompil/hands-on/blob/master/PySpark_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обзор модуля PySpark MLlib


Модуль MLlib предназначен для использования уже готовых алгоритмов машинного обучения в прикладных задачах при обработке больших данных. Во многом он схож с Scikit-learn, что делает его легким в освоении и позволяет переносить уже готовые методы обработки данных на платформу Spark.

Для лучшего понимания того, зачем делать определенные этапы потом, мы обсудим задачу машинного обучения  в первом приближении.

Вне зависимости от конкретного выбранного алгоритма анализа данных можно выделить схожие этапы:


*   Загрузка и отчистка данных.
*   Извлечение признаков (фич)
*   Тренировка модели
*   Оценка целевых показателей

Эти этапы могут повторяться неоднократно в процессе настройки гиперпараметров модели.

В Spark такой подход унифицировали и сосокупность таких этапов упаковали в **Pipeline**.
Каждый этап в Pipeline представляет из себя либо Transformer, либо Estimator, либо Evaluator.
При этом, Pipeline может содержать произвольное количество таких унафицированных этапов, каждый из которых может быть соединен с другим и выступать как източником , так и потребителем для других этапов.
Каждый из таких этапов надо рассмотреть подробнее.





**Transformer:** 


*   Используется на этапе препроцессинга
*   Преобразует данные в необходимый формат для далдьнейшей обработки
*   Обрабатывает по столбцам столбец на входе -> столбец на выходе

Примеры Transformers:
          

*   List item
*   List item



